# 2. Filter wavelengths

In the previous notebook, we retrieved a light curve of GW170817. In this notebook, we will use the [SVO Filter Profile Service](http://svo2.cab.inta-csic.es/svo/theory/fps/) to look up central wavelengths of the photometric bands.

The SVO Filter Profile Service is an exhaustive database of photometric response curves for many telescopes and instruments. In most cases, we are *not* using the response for the precise telescope and instrument, but rather a *representative* response. For example, we use the same central wavelength for the *K* band filter on every near-infrared instrument.

## Imports

In [1]:
from astropy.table import Table
from astropy.utils.data import download_file
from astropy import units as u
from lxml import etree

## Mapping between names of photometric bands in the Open Kilonova Catalog and the SVO Filter Service

In [2]:
filterdict = {
    'F110W': 'HST/WFC3_IR.F110W',
    'F160W': 'HST/WFC3_IR.F160W',
    'F336W': 'HST/WFC3_UVIS2.F336W',
    'F475W': 'HST/WFC3_UVIS2.F475W',
    'F606W': 'HST/WFC3_UVIS2.F606W',
    'F625W': 'HST/WFC3_UVIS2.F625W',
    'F775W': 'HST/WFC3_UVIS2.F775W',
    'F814W': 'HST/WFC3_UVIS2.F814W',
    'F850W': 'HST/WFC3_UVIS2.F850LP',
    'B': 'Swift/UVOT.B',
    'U': 'Swift/UVOT.U',
    'V': 'Swift/UVOT.V',
    'W1': 'Swift/UVOT.UVW1',
    'W2': 'Swift/UVOT.UVW2',
    'M2': 'Swift/UVOT.UVM2',
    'H': 'Gemini/Flamingos2.H',
    'I': 'CTIO/DECam.I',
    'Y': 'CTIO/DECam.Y',
    'R': 'CTIO/DECam.R',
    'J': 'Gemini/Flamingos2.J',
    'K': 'Gemini/Flamingos2.Ks',
    'Ks': 'Gemini/Flamingos2.Ks',
    'u': 'SLOAN/SDSS.u',
    'g': 'SLOAN/SDSS.g',
    'r': 'SLOAN/SDSS.r',
    'i': 'SLOAN/SDSS.i',
    'z': 'SLOAN/SDSS.z',
    'y': 'PAN-STARRS/PS1.y',
    'J1': 'LCO/FourStar.J1',
    'w': 'LasCumbres/LasCumbres.PS_w'
}

In [3]:
filter_names = []
filter_wavelengths = []

# Loop over all filters
for name, longname in filterdict.items():

    # Scrape the SVO service
    pattern = 'http://svo2.cab.inta-csic.es/svo/theory/fps3/fps.php?PhotCalID={}/AB&VERB=2'
    filename = download_file(pattern.format(longname), cache=True)

    # Parse XML to get the central wavelength in angstroms
    tree = etree.parse(filename)
    elem, = tree.iterfind(
        '//{*}PARAM[@name="WavelengthCen"][@unit="Angstrom"]')

    # Store the filter name and wavelength
    filter_names.append(name)
    filter_wavelengths.append(float(elem.attrib['value']))

# Build table.
filters = Table([filter_names, filter_wavelengths], names=['band', 'wavelength'])

# Sort by wavelength.
filters.sort('wavelength')

# Add unit metadata.
filters['wavelength'].unit = u.angstrom

# Show table.
filters

band,wavelength
,Angstrom
str5,float64
W2,1941.22132174
M2,2246.00356226
W1,2604.56916481
F336W,3368.34066118
U,3463.14480935
u,3565.05466331
B,4371.22288257
g,4700.32922202


## Save as ASCII

In [4]:
filters.write('data/filters.txt', format='ascii.ecsv', overwrite=True)